<a href="https://colab.research.google.com/github/hassssan051/portrait-video-synthesis/blob/audio-to-descriptor-pred/prediction/KNN_FirstFrame_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import zipfile
from tqdm import tqdm
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler

import torch.nn.functional as F
from sklearn.neighbors import NearestNeighbors
from collections import defaultdict
import pickle


In [ ]:
#loading MFCC features of RAVDESS dataset
datasetPath = 'DatasetForLSTM'
Zipped_inside_folder = 'RAVDESS_MFCC'
with zipfile.ZipFile('drive/MyDrive/'+Zipped_inside_folder+'.zip', 'r') as zip_ref:
    zip_ref.extractall(datasetPath)


In [ ]:
#getting descriptors of frames of each video and storing this information in a dictionary (true_descriptos) where key is video name and value is a list of descriptors of its frames.

clusters_info = 'Sawaiz_2/pkl_for_lstm_encoded/22_23_33_800'
file_path = '/content/drive/MyDrive/'+clusters_info+'/live_portrait_descriptors_all_encoder.pkl'

# Open the file in binary read mode and load the data
with open(file_path, 'rb') as file:
    data = pickle.load(file)

video_dict = defaultdict(list)


# Populate the video_dict with frame arrays in order
for key, value in data.items():
    # Split the key to extract video name and frame number
    parts = key.split('/')
    if 'M' not in key: #For Ravdess data
      video_name = parts[1]  # Extracts '02-01-01-01-02-02-16'
      frame_number = int(parts[2].split('.')[0])  # Extracts frame number as an integer (e.g., 1)

    else: #for MEAD
      video_name = parts[0] + "__" + parts[2] + "__" + parts[3] + "__" + parts[4]
      frame_number = int(parts[-1].split(".")[0].split("_")[-1])
    # Append the frame array to the respective video entry in the dictionary
    video_dict[video_name].append((frame_number, value))



# Sort frames for each video by frame number and concatenate them into a single array
final_video_dict = {}
for video_name, frames in video_dict.items():
    # Sort frames by frame number to ensure the order is correct
    sorted_frames = sorted(frames, key=lambda x: x[0])
    # Extract only the frame data, discarding the frame numbers
    sorted_arrays = [frame_data for _, frame_data in sorted_frames]
    # Concatenate all frames into a single numpy array
    final_video_dict[video_name] = np.vstack(sorted_arrays)
true_descriptors = final_video_dict
videos_list = list(true_descriptors.keys())
print(len(videos_list))

9282


In [ ]:
file_path = '/content/drive/MyDrive/'+clusters_info+'/averaged_descriptors_encoded.pkl'

with open(file_path, 'rb') as file:
    clusters_data = pickle.load(file)

# Actual labels for the LP
file_path = '/content/drive/MyDrive/'+clusters_info+'/frame_to_cluster_mapping.pkl'
with open(file_path, 'rb') as file:
    frames_data = pickle.load(file)

In [ ]:
for key, value in frames_data.items():
    sorted_value = sorted(value, key=lambda x: int(x[0].split('_')[-1].split('.')[0]) if '_' in x[0] else int(x[0].split('.')[0]))
    frames_data[key] = sorted_value

for key, val in frames_data.items():
    frames_data[key] = [ x[1] for x in val]

frames_data_new = {}
for key, val in frames_data.items():
    if 'M' in key:
      parts = key.split("/")
      video_name = parts[0] + "__" + parts[2] + "__" + parts[3] + "__" + parts[4]
      frames_data_new[video_name] = val
    else:
      frames_data_new[key] = val
frames_data = frames_data_new
#Here, frames_data is a dictionary where key is video name and value is list of cluster ids of its frames.


In [ ]:
number_of_clusters = 800 # Updated
stacked_descriptors = [clusters_data[val] for val in range(number_of_clusters)]
clusters_descriptors = np.vstack(stacked_descriptors) #A numpy array of cluster decriptors

In [ ]:
print(np.max(clusters_descriptors))

1.5142198


In [ ]:
#Here, key is a video name and value is a list of cluster representatives of those clusters to which its frames are mapped
clusters_rep_as_ground_truth_for_a_video = {}
for video, frames in frames_data.items():
  stacked_clusters_rep = [clusters_descriptors[val] for val in frames]
  clusters_rep_as_ground_truth_for_a_video[video] = np.vstack(stacked_clusters_rep)

In [ ]:
clusters_info = 'Sawaiz_2/pkl_for_lstm_encoded/23_25_38_800'
file_path = '/content/drive/MyDrive/'+clusters_info+'/live_portrait_descriptor_all_with_mead.pkl'
with open(file_path, 'rb') as file:
    des_raw = pickle.load(file)

In [ ]:
print(des_raw.keys())

Buffered data was truncated after reaching the output size limit.

In [ ]:
#print(des_raw[0][0]['driving_template_dct']['motion'][0]['x_s'].shape)
# kp = des_raw[0][0]['driving_template_dct']['motion'][0]['kp'].squeeze()
# exp = des_raw[0][0]['driving_template_dct']['motion'][0]['exp'].squeeze()
# r = des_raw[0][0]['driving_template_dct']['motion'][0]['R'].squeeze()
# t = des_raw[0][0]['driving_template_dct']['motion'][0]['t']
# s = des_raw[0][0]['driving_template_dct']['motion'][0]['scale'][0][0]

# x_s = des_raw[0][0]['driving_template_dct']['motion'][0]['x_s'].squeeze()
# x_s_ = s * (np.matmul(kp,r)+exp) + t


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

mead_mfcc_path = "/content/drive/MyDrive/MEAD_MFCC/"
# Load and prepare data
dataset_path = "DatasetForLSTM/"+ Zipped_inside_folder

# Update video paths
# Go through each video name and update path of the csv --> wave to vec csvs
all_video_paths = []
for video in videos_list:
  if 'M' in video: #For MEAD
    all_video_paths.append(mead_mfcc_path+video+".csv")
  else:
    all_video_paths.append("DatasetForLSTM/"+Zipped_inside_folder+"/"+video+".csv")

train_video_paths, test_video_paths = train_test_split(all_video_paths, test_size=0.05, random_state=42)

Using device: cuda


In [ ]:
def load_all_features_and_metadata(video_paths):
    all_features = []  # Store the features
    metadata = []  # Store the (video_index, row_index) for each feature

    for video_index, video_path in enumerate(video_paths):
        # Load the MFCC features from the CSV (assuming no header)
        df = pd.read_csv(video_path, header=None)
        video_name = video_path.split(".")[0].split("/")[-1]
        num_rows = true_descriptors[video_name].shape[0]
        features = df.iloc[:num_rows, :].values.astype(np.float32)
        #features = features.reshape(1,-1) #For first frame only
        # Store each feature along with its metadata (video_index, row_index)
        for row_index, feature in enumerate(features):
            all_features.append(feature)
            metadata.append((video_index, row_index))

    return np.array(all_features), metadata

all_features, metadata = load_all_features_and_metadata(all_video_paths)


In [ ]:
K =4
# Initialize the NearestNeighbors model
knn = NearestNeighbors(n_neighbors=K, metric='euclidean')
knn.fit(all_features)  # Fit on all features

NearestNeighbors(metric='euclidean', n_neighbors=2)

In [ ]:
# Function to find K nearest neighbors
def find_knn(query_feature,current_video_path,  video_paths,knn, k=1):
    # Query the nearest neighbors for the given feature
    distances, indices = knn.kneighbors([query_feature])
    # Prepare the results (video_index, row_index)
    neighbors = []
    for i in range(k):
        # For each nearest neighbor
        video_index, row_index = metadata[indices[0][i]]
        if video_paths[video_index] == current_video_path and row_index == 0:
          continue
        neighbors.append({
            'video': video_paths[video_index],  # Video file path
            'row_index': row_index,             # Row index in the video CSV
            'distance': distances[0][i]         # Distance to the query feature
        })
        if len(neighbors) == k-1:
          break

    return neighbors

In [ ]:
def get_predicted_labels(outputs, cluster_descriptors, device):
    # Ensure cluster_descriptors is a torch tensor and move to device
    cluster_descriptors = torch.tensor(cluster_descriptors, device=device,  dtype=torch.float32)
    #print(cluster_descriptors.shape)
    # Initialize an empty list to store predicted labels for each frame
    predicted_labels = []
    #outputs = outputs.squeeze(0)
    # Iterate over each frame descriptor in outputs
    for frame_descriptor in outputs:
        # Calculate Euclidean distances between the frame descriptor and each cluster descriptor
        #print(frame_descriptor.shape)

        distances = torch.norm(cluster_descriptors - frame_descriptor, dim=1)

        # Find the index of the minimum distance (i.e., closest cluster descriptor)
        predicted_label = torch.argmin(distances)
        predicted_labels.append(predicted_label.item())

    # Convert predicted labels list to a tensor on the same device
    return predicted_label.detach().cpu().numpy().tolist()

In [ ]:
# for i_i in [0,1,2]:
#   print(i_i)

#Getting K-nearest neigbors and Averaging their descriptors.
matched_train = 0
total_train = 0
for video_path in train_video_paths:
  df = pd.read_csv(video_path, header=None)
  features = df.iloc[0, :].values.astype(np.float32)
  result = find_knn(features,video_path, all_video_paths, knn, K)
  descriptor_list = []
  for i in range(len(result)):
    video_name = result[i]['video'].split(".")[0].split("/")[-1]
    row_idx = result[i]['row_index']
    try:
      descriptor = clusters_descriptors[frames_data[video_name][row_idx]]
    except:
      print(video_name)
      print(row_idx)
      raise ValueError
    descriptor_list.append(descriptor)
  descriptor_list = np.vstack(descriptor_list)
  #descriptor_list = descriptor_list[i_i]
  #descriptor_list = descriptor_list.reshape(1,-1)

  descriptor_list = np.mean(descriptor_list, axis=0, keepdims=True)
  descriptor_list = torch.tensor(descriptor_list, device=device,  dtype=torch.float32)
  predicted_labels = get_predicted_labels(descriptor_list, clusters_descriptors, device)
  video_name = video_path.split(".")[0].split("/")[-1]
  original_label = frames_data[video_name][0]
  if original_label == predicted_labels:
    matched_train +=1
  total_train +=1
print("Accuracy on train:", (matched_train*100)/total_train,"%")

matched_test = 0
total_test = 0
for video_path in test_video_paths:
  df = pd.read_csv(video_path, header=None)
  features = df.iloc[0, :].values.astype(np.float32)
  result = find_knn(features,video_path, all_video_paths, knn, K)
  descriptor_list = []
  for i in range(len(result)):
    video_name = result[i]['video'].split(".")[0].split("/")[-1]
    row_idx = result[i]['row_index']
    descriptor = clusters_descriptors[frames_data[video_name][row_idx]]
    descriptor_list.append(descriptor)
  descriptor_list = np.vstack(descriptor_list)
  #descriptor_list = descriptor_list[i_i]
  #descriptor_list = descriptor_list.reshape(1,-1)
  descriptor_list = np.mean(descriptor_list, axis=0, keepdims=True)
  descriptor_list = torch.tensor(descriptor_list, device=device,  dtype=torch.float32)
  predicted_labels = get_predicted_labels(descriptor_list, clusters_descriptors, device)
  video_name = video_path.split(".")[0].split("/")[-1]
  original_label = frames_data[video_name][0]
  if original_label == predicted_labels:
    matched_test +=1
  total_test +=1

print("Accuracy on test:", (matched_test*100)/total_test,"%")


print("Accuracy on Test + Train:", ((matched_test + matched_train)*100)/(total_test+total_train),"%")




Accuracy on train: 8.72178745605081 %
Accuracy on test: 7.741935483870968 %
Accuracy on Test + Train: 8.672699849170437 %


In [ ]:
'''
on KNN, with N=3
Overall acc is 17%
N= 0 gives acc 17.8%
N= 1 gives acc 19.9%
N= 2 gives acc 18.2%
'''

In [ ]:
# mead_videos = 0
# ravdess_videos = 0
# for i in videos_list:
#   if 'M' in i:
#     mead_videos+=1
#   else:
#     ravdess_videos+=1
# print(mead_videos)
# print(ravdess_videos)

6671
2611
